# **A Little about Python and Jupyter**


**Additional Reading**
1. [Scientific Python Introduction](http://hplgit.github.io/bumpy/doc/pub/basics.html)

Recommended [Jupyter Theme](https://github.com/dunovank/jupyter-themes) for presenting this notebook:
````
jt -t grade3 -cellw=90% -fs=20 -tfs=20 -ofs=20
````

![](../graphics/spacer_small.png)

** Reusing Code**
- Often, we will want to reuse methods from one notebook to the next.
- We could copy + paste these methods into this notebook, but a better appraoch is to create a python module in a seperate .py file that holds commonly used code. 
- You may have noticed the "util" directory in this repo:

In [3]:
ls ..

README.md         data/             notebooks/        util/
__MACOSX/         deprecated/       papers/           videos/
challenge/        graphics/         requirements.txt


It contains a serveral python modules that contain the various methods we'll develop in the module:

In [5]:
ls ../util/*

../util/filters.py         ../util/image.py
../util/get_and_unpack.py

../util/__pycache__:
filters.cpython-36.pyc         image.cpython-36.pyc
get_and_unpack.cpython-36.pyc


- Let's go have a look at one of these .py files in a text editor. 
- Here's how to import sepcific methods from .py files. 

In [17]:
# First, since our util directory is in our top-level directory, to access 
## these files, we need to add the top-level repo direcotry to our path.
import sys
sys.path.append('..')

In [18]:
#We can now import methods from our util directory
from util.image import convert_to_grayscale

While we're at it, let's look a few nice notebook features. We can see the docstring of the methods we're using right in the notebook by adding a question mark to the end of method or class names:

In [19]:
convert_to_grayscale?

![](../graphics/spacer_small.png)

**Some nice notebook features**

Another nice feature: Press **Shift+Tab** with your cursor inside a methods parenthesis to see it's arguments. 

![](graphics/spacer_small.png)

![](../videos/shift_tab.gif)

![](graphics/spacer_small.png)

And finally, you can access search all commands using the command pallate:
- **(cmnd + shift + p)** on mac
- **(ctrl + shift + p)** on linux/windows

And to view all keyboard shortucts press **esc** then **h.**

![](../videos/jupyter_shortcuts.gif)

**One more thing about importing python modules**

- Modules are compiled on import. 
- If we poke around our util director after import, we see that each python file is compiled into it's own .pyc in a python automatically generated directory called \__pycache__

In [21]:
ls ../util*

__pycache__/       filters.py         get_and_unpack.py  image.py


In [22]:
ls ../util/__pycache__/

filters.cpython-36.pyc         image.cpython-36.pyc
get_and_unpack.cpython-36.pyc


- The important thing to note here is that if you make a change to a .py file, the results of this change will not actually show up **until you reimport the module**, recompiling the .py into .pyc.
- Generally, I'll just run "Restart and Run All" from the Kernel dropdown menu when I make a module change. 
- **Quick Demo:**

In [24]:
#import a test image:
im = imread('../data/medium_2/ball/ball_3.jpg')

In [25]:
gray = convert_to_grayscale(im/255.)

![](graphics/spacer_small.png)

![](../graphics/spacer_small.png)

** A little bit about scientific python**

Python is an incredibly tool for science + engineering. It's one of the easiest languages to learn, but incredibly powerful, especially thanks to packages like numpy, libraries such as opencv, and frameworks like tensorflow. There are lots of great resources on python and scientific python out there - these are terrific. If your new or learning python, I also encourage you to just try things. Python is a friendly enough language that usually after hacking around a bit, plus some help from google and stack overflow, you can figure out what you need.